<a href="https://colab.research.google.com/github/iynnn/Data-Science/blob/main/Gamma%20Fest%202023/Percobaan_ke_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
path =  "/content/drive/MyDrive/[02] Lomba/[Current] GAMMAFEST IPB 2023 /Penyisihan"
df_train_ = pd.read_csv(path + "/train.csv")
df_test = pd.read_csv(path + "/test.csv")

Mounted at /content/drive


In [ ]:
# train = df_train.drop(columns = ["id", "DC235", "DC242"], axis = 1)
# test = df_test.drop(columns = ["id", "DC235", "DC242"], axis = 1)
train = df_train.drop(columns = ["id",  'DC235', "DC242", "DC230b", "DC232", "DC232b"], axis = 1)
test = df_test.drop(columns = ["id",  'DC235', "DC242", "DC230b", "DC232", "DC232b"], axis = 1)

# Cek Konsistensi
seperti yang dijelasin erza kemarin, kalau misal kita mau ngecek konsistensi apakah yang ada di training itu udah semuanya ada di metadata, apakah ada kejanggalan berupa ada nilai kategorik yang tidak ada di metadata

In [ ]:
# Memasukkan inputan data metadata
list_var = pd.read_excel(path + "/Metadata DSC.xlsx", sheet_name = "Sheet1")
list_var = list_var.drop(columns =  [ 'DC235', "DC242", "DC230b", "DC232", "DC232b"])
list_var.head()

,DC025,DC024,DC205,DC206,DC207,DC208,DC209,DC210,DC211,DC212,...,DC237d,DC237e,DC237f,DC241,DC244,DC246,DC252,DC270a,DC109,DC142a
0,1.0,11,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,11.0,998.0
1,2.0,12,11.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,12.0,NaN
2,NaN,13,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.0,8.0,8.0,3.0,NaN,NaN,2.0,3.0,21.0,NaN
3,NaN,14,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,6.0,NaN,NaN,3.0,4.0,31.0,NaN
4,NaN,15,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,5.0,32.0,NaN


In [ ]:
# Memasukkan count value pada setiap variabel ini pada data training
for c in train.columns :
  globals()[f'count_{c}'] = pd.DataFrame(train[c].value_counts())

In [ ]:
def cek(sc1, sc2):
  # list 1 ini nanti isinya yang dari dataframe asli
  # list 2 ini yang dari metadata

  list1 = list(sc1.index)

  list2 = list_var[sc2].values
  list2 = list2[~np.isnan(list2)]
  list2 = list2.tolist()

  c = []
  for number in list1:
    if  number not in list2:
      if number not in c:
        c.append(number)

  if not c:
    print("Tidak ada masalah di variabel ", sc2)
  else:
    print("Terdapat nilai yang tidak ada dalam metadata pada variabel ", sc2,  " yaitu : {}". format(c))
  # print((sc1))


# Mengecek pada masing-masing variabel data training apakah sudah konsisten apa belom
for var in list(list_var.columns):
  sc1 = "count_" + var
  sc1 = globals()[sc1]

  sc2 = var

  cek(sc1, sc2)


Tidak ada masalah di variabel  DC025
Tidak ada masalah di variabel  DC024
Tidak ada masalah di variabel  DC205
Tidak ada masalah di variabel  DC206
Tidak ada masalah di variabel  DC207
Tidak ada masalah di variabel  DC208
Tidak ada masalah di variabel  DC209
Tidak ada masalah di variabel  DC210
Tidak ada masalah di variabel  DC211
Tidak ada masalah di variabel  DC212
Tidak ada masalah di variabel  DC213
Tidak ada masalah di variabel  DC214
Tidak ada masalah di variabel  DC215
Terdapat nilai yang tidak ada dalam metadata pada variabel  DC216  yaitu : [2.0, 3.0, 1.0, 4.0, 5.0, 0.0, 6.0, 7.0, 8.0, 10.0, 9.0, 20.0, 11.0, 12.0, 14.0]
Tidak ada masalah di variabel  DC217
Tidak ada masalah di variabel  DC219
Terdapat nilai yang tidak ada dalam metadata pada variabel  DC220  yaitu : [47.0, 52.0, 42.0, 48.0, 45.0, 44.0, 41.0, 46.0, 37.0, 49.0, 40.0, 43.0, 57.0, 51.0, 54.0, 53.0, 39.0, 38.0, 34.0, 35.0, 36.0, 50.0, 55.0, 60.0, 58.0, 59.0, 56.0, 33.0, 62.0, 32.0, 31.0, 61.0, 63.0, 65.0, 30.0, 64.

# Imputasi dengan Random Forest

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier


imp_num = IterativeImputer(estimator=RandomForestRegressor(),
                            initial_strategy='mean',
                            max_iter=10, random_state=42)

imp_cat = IterativeImputer(estimator=RandomForestClassifier(),
                            initial_strategy='most_frequent',
                            max_iter=5, random_state=42)


In [ ]:
# Pisahkan dataset yang punya nilai other pada variabel numerik
cat = list(train.drop(columns = ["DC216", "DC220", "DC142a"], axis = 1))
num = list(train[["DC216", "DC220", "DC142a"]])

In [ ]:
train[num].describe()

,DC216,DC220,DC142a
count,35934.000000,35969.000000,35921.000000
mean,2.323732,48.475660,82.668216
std,1.105251,13.713263,91.194377
min,0.000000,13.000000,0.000000
25%,2.000000,38.000000,42.000000
50%,2.000000,48.000000,63.000000
75%,3.000000,58.000000,96.000000
max,20.000000,98.000000,998.000000


In [ ]:
train_1 = train
# train_1.query('DC142a == 0 or DC142a == 998'), numpy.NaN)
train_1 = train_1.replace({'DC142a' : {998 : np.NaN, 0 : np.NaN}, 'DC216' : {0 : np.NaN}, 'DC220' : {98 : np.NaN}})
# train_1.query('DC142a == 998')
# train_1 = train_1.drop(columns = ['DC235', "DC242"], axis = 1)
train_1.isna().sum()

# ini menambah nilai NA yang ada pada variabel DC142a

DC024       0
DC025       0
DC205      11
DC206      21
DC207      44
DC208      13
DC209      18
DC210      41
DC211      24
DC212      24
DC213      77
DC214       9
DC215      14
DC216     285
DC217       0
DC219       0
DC220       5
DC226       6
DC230a     11
DC237      21
DC237a     37
DC237b     37
DC237c     37
DC237d     37
DC237e     37
DC237f     37
DC241     612
DC244       4
DC246       9
DC252      14
DC270a      0
DC109      11
DC142a    268
DC201       3
dtype: int64

## Imputasi Numerik

In [ ]:
train_imp = train_1
train_imp[num] = imp_num.fit_transform(train_1[num])

/usr/local/lib/python3.10/dist-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [ ]:
train_imp[num].describe()

,DC216,DC220,DC142a
count,35973.000000,35973.000000,35973.000000
mean,2.335004,48.474348,77.359405
std,1.090530,13.710037,58.462207
min,1.000000,13.000000,1.000000
25%,2.000000,38.000000,42.000000
50%,2.000000,48.000000,63.000000
75%,3.000000,58.000000,96.000000
max,20.000000,95.000000,995.000000


## Imputasi Kategorik

In [ ]:
train_1["DC201"] = train_1["DC201"].map({"Layak Minum" : 1, "Tidak Layak Minum" : 0})

In [ ]:
train_batch = train_1
tiga = math.floor(len(train_1)/3)
train_b1 = train_batch.iloc[:tiga,]
train_b2 = train_batch.iloc[tiga:2*tiga,]
train_b3 = train_batch.iloc[2*tiga:,]


ncat  = math.floor(len(cat)/3)
cat1 = cat[:ncat]
cat2 = cat[ncat:2*ncat]
cat3 = cat[2*ncat:]


In [ ]:
# Batch 1
train_b1[cat1] = imp_cat.fit_transform(train_b1[cat1])
train_b1[cat2] = imp_cat.fit_transform(train_b1[cat2])
train_b1[cat3] = imp_cat.fit_transform(train_b1[cat3])

<ipython-input-15-3b22a756a666>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_b1[cat1] = imp_cat.fit_transform(train_b1[cat1])
<ipython-input-15-3b22a756a666>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_b1[cat2] = imp_cat.fit_transform(train_b1[cat2])
<ipython-input-15-3b22a756a666>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

In [ ]:
# Batch 2
train_b2[cat1] = imp_cat.fit_transform(train_b2[cat1])
train_b2[cat2] = imp_cat.fit_transform(train_b2[cat2])
train_b2[cat3] = imp_cat.fit_transform(train_b2[cat3])

<ipython-input-16-f9887fb91865>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_b2[cat1] = imp_cat.fit_transform(train_b2[cat1])
<ipython-input-16-f9887fb91865>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_b2[cat2] = imp_cat.fit_transform(train_b2[cat2])
<ipython-input-16-f9887fb91865>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

In [ ]:
# Batch 3
train_b3[cat1] = imp_cat.fit_transform(train_b3[cat1])
train_b3[cat2] = imp_cat.fit_transform(train_b3[cat2])
train_b3[cat3] = imp_cat.fit_transform(train_b3[cat3])

/usr/local/lib/python3.10/dist-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
<ipython-input-17-a0c06202097d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_b3[cat1] = imp_cat.fit_transform(train_b3[cat1])
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
<ipython-input-17-a0c06202097d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

In [ ]:
# train_b1[cat] = imp_cat.fit_transform(train_b1[cat])
# train_b2[cat] = imp_cat.fit_transform(train_b2[cat])
# train_b3[cat] = imp_cat.fit_transform(train_b3[cat])

In [ ]:
train_imputasi = pd.concat([train_b1, train_b2,train_b3])
train_imputasi.isna().sum()

DC024     0
DC025     0
DC205     0
DC206     0
DC207     0
DC208     0
DC209     0
DC210     0
DC211     0
DC212     0
DC213     0
DC214     0
DC215     0
DC216     0
DC217     0
DC219     0
DC220     0
DC226     0
DC230a    0
DC237     0
DC237a    0
DC237b    0
DC237c    0
DC237d    0
DC237e    0
DC237f    0
DC241     0
DC244     0
DC246     0
DC252     0
DC270a    0
DC109     0
DC142a    0
DC201     0
dtype: int64

In [ ]:
# train_imputasi.to_csv(path + "/imputasi kedua.csv", index = False)

# Imputation Handling with Mice

In [ ]:
!pip install miceforest --no-cache-dir
!pip install git+https://github.com/AnotherSamWilson/miceforest.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/AnotherSamWilson/miceforest.git to /tmp/pip-req-build-3i1cwb6m
  Running command git clone --filter=blob:none --quiet https://github.com/AnotherSamWilson/miceforest.git /tmp/pip-req-build-3i1cwb6m
  Resolved https://github.com/AnotherSamWilson/miceforest.git to commit d9359a89204e3b5f10cc02e7e621a22c213e5453
  Preparing metadata (setup.py) ... done


In [ ]:
import miceforest as mfst
# Create kernel.
kds = mfst.ImputationKernel(
  train_1,
  save_all_iterations=True,
  random_state=100
)

# Run the MICE algorithm for 5 iterations
kds.mice(5)

In [ ]:
df_imputed = kds.complete_data()
df_imputed.head()

,DC024,DC025,DC205,DC206,DC207,DC208,DC209,DC210,DC211,DC212,...,DC237e,DC237f,DC241,DC244,DC246,DC252,DC270a,DC109,DC142a,DC201
0,11,2,16.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,2.0,1,12.0,60.0,1.0
1,11,2,31.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,2,31.0,70.0,1.0
2,11,2,12.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,4,11.0,75.0,1.0
3,11,2,12.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,2.0,1.0,0.0,1.0,1,11.0,85.0,1.0
4,11,2,12.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,2,11.0,65.0,1.0


In [ ]:
df_imputed.isna().sum()

DC024     0
DC025     0
DC205     0
DC206     0
DC207     0
DC208     0
DC209     0
DC210     0
DC211     0
DC212     0
DC213     0
DC214     0
DC215     0
DC216     0
DC217     0
DC219     0
DC220     0
DC226     0
DC230a    0
DC237     0
DC237a    0
DC237b    0
DC237c    0
DC237d    0
DC237e    0
DC237f    0
DC241     0
DC244     0
DC246     0
DC252     0
DC270a    0
DC109     0
DC142a    0
DC201     0
dtype: int64

In [ ]:
# df_imputed.to_csv(path + "/imputasi mice5.csv", index = False)

# Imbalance Data Set

### Split data

In [ ]:
from sklearn.model_selection import train_test_split

X = df_imputed.drop("DC201", axis = 1)
Y = df_imputed.DC201

# split into 80:20 ration
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)



X_res, y_res = sm.fit_resample(X_train, y_train)

In [ ]:
len(y_res)

51838

In [ ]:
y_res = pd.DataFrame(y_res)
y_res.value_counts()

DC201
0.0      25919
1.0      25919
dtype: int64

# Modeling

In [ ]:
import xgboost as xgb

xgb_cl = xgb.XGBClassifier()

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

# Init classifier
xgb_clf = xgb.XGBClassifier()

# Fit
xgb_clf.fit(X_res, y_res)

# Predict
preds = xgb_clf.predict(X_test)

accuracy_score(y_test, preds)

0.9135510771369006

In [ ]:
f1_score(y_test, preds)

0.9532752403846154

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.62      0.32      0.42       713
         1.0       0.93      0.98      0.95      6482

    accuracy                           0.91      7195
   macro avg       0.78      0.65      0.69      7195
weighted avg       0.90      0.91      0.90      7195



In [ ]:
print(confusion_matrix(y_test, preds))

[[ 228  485]
 [ 137 6345]]


In [ ]:
pred_xgb = xgb_clf.predict(test)

In [ ]:
pred_xgb = pd.DataFrame(pred_xgb)
pred_xgb.value_counts()

1    11441
0      549
dtype: int64

In [ ]:
pred_xgb["id"] = df_test["id"]
pred_xgb["DC201"] = pred_xgb[0].map({1 : "Layak Minum", 0: "Tidak Layak Minum"})
pred_xgb = pred_xgb[["id", "DC201"]]
pred_xgb

,id,DC201
0,26718,Layak Minum
1,26802,Layak Minum
2,41302,Layak Minum
3,38698,Layak Minum
4,44257,Layak Minum
...,...,...
11985,36943,Layak Minum
11986,33415,Layak Minum
11987,41998,Layak Minum
11988,41567,Layak Minum


In [ ]:
# pred_xgb.to_csv(path + "/pred xgb baru .csv", index = False)

In [ ]:
test_1 = test
# test_1.query('DC142a == 0 or DC142a == 998'), numpy.NaN)
test_1 = test_1.replace({'DC142a' : {998 : np.NaN, 0 : np.NaN}, 'DC216' : {0 : np.NaN}, 'DC220' : {98 : np.NaN}})
# test_1.query('DC142a == 998')
# test_1 = test_1.drop(columns = ['DC235', "DC242"], axis = 1)
test_1.isna().sum()

# ini menambah nilai NA yang ada pada variabel DC142a

DC024      0
DC025      0
DC205      0
DC206      0
DC207      0
DC208      0
DC209      0
DC210      0
DC211      0
DC212      0
DC213      0
DC214      0
DC215      0
DC216     61
DC217      0
DC219      0
DC220      0
DC226      0
DC230a     0
DC237      0
DC237a     0
DC237b     0
DC237c     0
DC237d     0
DC237e     0
DC237f     0
DC241      0
DC244      0
DC246      0
DC252      0
DC270a     0
DC109      0
DC142a    60
dtype: int64

In [ ]:
test_imp = test_1
test_imp[num] = imp_num.fit_transform(test_1[num])

/usr/local/lib/python3.10/dist-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [ ]:
pred_xgb2 = pd.DataFrame(xgb_clf.predict(test_imp))
pred_xgb2["id"] = df_test["id"]
pred_xgb2["DC201"] = pred_xgb2[0].map({1 : "Layak Minum", 0: "Tidak Layak Minum"})
pred_xgb2 = pred_xgb2[["id", "DC201"]]
pred_xgb2["DC201"].value_counts()

Layak Minum          11428
Tidak Layak Minum      562
Name: DC201, dtype: int64

In [ ]:
pred_xgb2.to_csv(path + "/pred xgb baru 2.csv", index = False)

## GRID SEARCH

In [ ]:
param_grid = {
    "max_depth": [3, 5, 10],
    "learning_rate": [0.1, 0.01],
    "gamma": [0, 0.25,  1],
    "reg_lambda": [0, 1, 10],
    "scale_pos_weight": [0, 3, 5],
    "subsample": [0.8],
    "colsample_bytree": [0.5],
}

parameters = {
    'max_depth': range (2, 10, 1),
    'n_estimators': range(60, 220, 40),
    'learning_rate': [0.1, 0.01, 0.05]
}

In [ ]:
from sklearn.model_selection import GridSearchCV

# Init classifier
xgb_cl = xgb.XGBClassifier(objective="binary:logistic")

# Init Grid Search
grid_cv = GridSearchCV(xgb_cl, parameters, n_jobs=-1, cv=3, scoring="roc_auc")

# Fit
xgb_new = grid_cv.fit(X_res, y_res)

In [ ]:
xgb_new.best_params_

{'learning_rate': 0.05, 'max_depth': 9, 'n_estimators': 140}

In [ ]:
# Predict
xgb_preds = xgb_new.predict(X_test)

accuracy_score(y_test, xgb_preds)

0.9125781792911745

In [ ]:
f1_score(y_test, xgb_preds)

0.9529367751589973

In [ ]:
print(classification_report(y_test, xgb_preds))

              precision    recall  f1-score   support

         0.0       0.63      0.28      0.39       713
         1.0       0.93      0.98      0.95      6482

    accuracy                           0.91      7195
   macro avg       0.78      0.63      0.67      7195
weighted avg       0.90      0.91      0.90      7195



In [ ]:
print(confusion_matrix(y_test, xgb_preds))

[[ 198  515]
 [ 114 6368]]


In [ ]:
pred_xgb_tune = pd.DataFrame(xgb_new.predict(test_imp))
pred_xgb_tune["id"] = df_test["id"]
pred_xgb_tune["DC201"] = pred_xgb_tune[0].map({1 : "Layak Minum", 0: "Tidak Layak Minum"})
pred_xgb_tune = pred_xgb_tune[["id", "DC201"]]
pred_xgb_tune["DC201"].value_counts()

Layak Minum          11504
Tidak Layak Minum      486
Name: DC201, dtype: int64

In [ ]:
pred_xgb_tune.to_csv(path + "/pred xgb baru tune.csv", index = False)

## Random Forest

In [ ]:
rf = RandomForestClassifier(random_state = 42)
rf.fit(X_res, y_res)
rf.get_params()

<ipython-input-132-214bddf98b00>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_res, y_res)


{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [ ]:
# Predict
pred_rf = rf.predict(X_test)

print(accuracy_score(y_test, pred_rf))
print(f1_score(y_test, pred_rf))
print(classification_report(y_test, pred_rf))

0.908825573314802
0.9510228460504704
              precision    recall  f1-score   support

         0.0       0.60      0.24      0.34       713
         1.0       0.92      0.98      0.95      6482

    accuracy                           0.91      7195
   macro avg       0.76      0.61      0.65      7195
weighted avg       0.89      0.91      0.89      7195



In [ ]:
pred_rf_biasa = pd.DataFrame(rf.predict(test_1))
pred_rf_biasa.value_counts()

1.0    11566
0.0      424
dtype: int64

## Tuning RF

In [ ]:
param_grid_rf = {
    'n_estimators': [100, 200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

In [ ]:
CV_rfc = GridSearchCV(estimator=rf, param_grid=param_grid_rf, cv= 5)
CV_rfc.fit(X_res, y_res)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: Fut

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [4, 5, 6, 7, 8],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [100, 200, 500]})

In [ ]:
CV_rfc.best_params_

{'criterion': 'gini',
 'max_depth': 8,
 'max_features': 'auto',
 'n_estimators': 200}

In [ ]:
pred_val_cv = CV_rfc.predict(X_test)
print(accuracy_score(y_test, pred_val_cv))
print(f1_score(y_test, pred_val_cv))
print(classification_report(y_test, pred_val_cv))

0.8592077831827658
0.9195840279431611
              precision    recall  f1-score   support

         0.0       0.36      0.55      0.44       713
         1.0       0.95      0.89      0.92      6482

    accuracy                           0.86      7195
   macro avg       0.65      0.72      0.68      7195
weighted avg       0.89      0.86      0.87      7195



In [ ]:
pred_rf_tune =  pd.DataFrame(CV_rfc.predict(test_1))
pred_rf_tune.value_counts()

1.0    10412
0.0     1578
dtype: int64

In [ ]:
pred_rf_tune2 = pd.DataFrame(CV_rfc.predict(test_imp))
pred_rf_tune2["id"] = df_test["id"]
pred_rf_tune2["DC201"] = pred_rf_tune2[0].map({1 : "Layak Minum", 0: "Tidak Layak Minum"})
pred_rf_tune2 = pred_rf_tune2[["id", "DC201"]]
pred_rf_tune2["DC201"].value_counts()

Layak Minum          10412
Tidak Layak Minum     1578
Name: DC201, dtype: int64

In [ ]:
pred_rf_tune2.to_csv(path + "/pred Random Forest baru tune.csv", index = False)

In [ ]:
rfc1=RandomForestClassifier(random_state=42, max_features='auto', n_estimators= 200, max_depth=10, criterion='gini')

In [ ]:
rfc1.fit(X_res, y_res)

<ipython-input-221-1e050ea155fd>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc1.fit(X_res, y_res)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


RandomForestClassifier(max_depth=10, max_features='auto', n_estimators=200,
                       random_state=42)

In [ ]:
pred_rf_tune1 = pd.DataFrame(rfc1.predict(test_imp))
pred_rf_tune1["id"] = df_test["id"]
pred_rf_tune1["DC201"] = pred_rf_tune1[0].map({1 : "Layak Minum", 0: "Tidak Layak Minum"})
pred_rf_tune1 = pred_rf_tune1[["id", "DC201"]]
pred_rf_tune1["DC201"].value_counts()

Layak Minum          10810
Tidak Layak Minum     1180
Name: DC201, dtype: int64

In [ ]:
pred_rf_coba = rfc1.predict(X_test)
print(accuracy_score(y_test, pred_rf_coba))
print(f1_score(y_test, pred_rf_coba))
print(classification_report(y_test, pred_rf_coba))

0.8800555941626129
0.9328665888759239
              precision    recall  f1-score   support

         0.0       0.41      0.47      0.44       713
         1.0       0.94      0.93      0.93      6482

    accuracy                           0.88      7195
   macro avg       0.67      0.70      0.69      7195
weighted avg       0.89      0.88      0.88      7195



In [ ]:
pred_rf_tune1.to_csv(path + "/pred Random Forest bismillah 10 max depth.csv", index = False)

In [ ]:
X_all = pd.concat([X_res, X_test])
y_test = pd.DataFrame(y_test)
Y_all = pd.concat([y_res, y_test])

In [ ]:
All = X_all
All["Y"] = Y_all

All.to_csv(path + "Data Insyaallah bersih udah di Smote juga.csv", index = False)

In [ ]:
# from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate
# scores = cross_validate(CV_rfc, X_all, Y_all , cv=5,
# scoring=('accuracy', 'f1'), return_train_score=True)
# predictions = cross_val_predict(CV_rfc, X_all, Y_all,
# cv=5)

# skor = pd.DataFrame(scores)
# skor

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: Fut

KeyboardInterrupt: ignored

In [ ]:
# Tuning di max depth aja
param_grid_rf2 = {
    'n_estimators': [200],
    'max_features': ['auto'],
    'max_depth' : [8, 9, 10,11],
    'criterion' :['gini']
}

In [ ]:
CV_rfc2 = GridSearchCV(estimator=rf, param_grid=param_grid_rf2, cv= 5)
CV_rfc2.fit(X_res, y_res)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: Fut

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'criterion': ['gini'], 'max_depth': [8, 9, 10, 11],
                         'max_features': ['auto'], 'n_estimators': [200]})

In [ ]:
CV_rfc2.best_params_

{'criterion': 'gini',
 'max_depth': 11,
 'max_features': 'auto',
 'n_estimators': 200}

In [ ]:
pred_val_cv1 = CV_rfc2.predict(X_test)
print(accuracy_score(y_test, pred_val_cv1))
print(f1_score(y_test, pred_val_cv1))
print(classification_report(y_test, pred_val_cv1))

0.8861709520500347
0.9367421024175485
              precision    recall  f1-score   support

         0.0       0.43      0.44      0.43       713
         1.0       0.94      0.94      0.94      6482

    accuracy                           0.89      7195
   macro avg       0.68      0.69      0.68      7195
weighted avg       0.89      0.89      0.89      7195



In [ ]:
print(confusion_matrix(y_test, pred_val_cv1))

[[ 312  401]
 [ 418 6064]]


In [ ]:
pred_rf_tune_lagi =  pd.DataFrame(CV_rfc2.predict(test_1))
pred_rf_tune_lagi.value_counts()

1.0    10943
0.0     1047
dtype: int64

In [ ]:
pred_rf_tune_lagi = pd.DataFrame(CV_rfc2.predict(test_imp))
pred_rf_tune_lagi["id"] = df_test["id"]
pred_rf_tune_lagi["DC201"] = pred_rf_tune_lagi[0].map({1 : "Layak Minum", 0: "Tidak Layak Minum"})
pred_rf_tune_lagi = pred_rf_tune_lagi[["id", "DC201"]]
pred_rf_tune_lagi["DC201"].value_counts()

Layak Minum          10943
Tidak Layak Minum     1047
Name: DC201, dtype: int64

In [ ]:
pred_rf_tune_lagi.to_csv(path + "/pred Random Forest baru tune lagi.csv", index = False)